In [2]:
import time
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pytz


In [169]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")

# Specify the path to the chromedriver executable
chrome_driver_path = "C:/Users/ddadmin/Desktop/tools/chromedriver-win64/chromedriver.exe"  # chromedriver path

# Set up the WebDriver with Chrome options
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Now you can access the already opened Chrome session
print(driver.title)  # Print the title of the currently opened page

Hodinkee Japan: ホディンキー / 時計メディア(@hodinkeejapan) • Instagram写真と動画


In [170]:
window_handles = driver.window_handles
for handle in window_handles:
    # Switch to the tab
    driver.switch_to.window(handle)
    
    # Check if the current tab is Instagram by inspecting the URL or title
    if "instagram.com" in driver.current_url:
        print("Found Instagram session")
        print(f"Instagram URL: {driver.current_url}")
        break
    else:
        print(f"Other tab URL: {driver.current_url}")

# Now you can interact with the Instagram session
# Example: Print the page title
print("Page title:", driver.title)

Found Instagram session
Instagram URL: https://www.instagram.com/hodinkeejapan/
Page title: Hodinkee Japan: ホディンキー / 時計メディア(@hodinkeejapan) • Instagram写真と動画


In [171]:
# mainDiv = driver.find_element(By.XPATH,"//main[@role='main']")
# print(mainDiv)

<selenium.webdriver.remote.webelement.WebElement (session="08b6d36ef2f998a58286d6c081030449", element="f.E9B9EEF6F7ED992CEC48A161464A49BF.d.EED170165D8A51EDE026304405762BAA.e.99573")>


In [172]:
# post_links = []
# mainChildContainer = mainDiv.find_elements(By.XPATH,"./div") 
# print(len(mainChildContainer))
# postContainer = mainChildContainer[0].find_elements(By.XPATH,"./div")[-1]
# postStyleDiv = postContainer.find_elements(By.XPATH,"./div")[0]
# postRowContainer = postStyleDiv.find_elements(By.XPATH,"./div") 
# print(len(postRowContainer))

# for div_row in postRowContainer:
#     divs = div_row.find_elements(By.XPATH, "div")
#     # print(f"Number of <div> elements in each row: {len(divs)}")
#     for div in divs:
#         a_tags = div.find_elements(By.TAG_NAME, "a")
#         if len(a_tags) > 0:
#             a_tag = a_tags[0]
#             href = a_tag.get_attribute("href")
#             post_links.append(href)
# print(post_links)
# print(len(post_links))


1
4
['https://www.instagram.com/p/DALSUyhveyl/', 'https://www.instagram.com/p/DAM0D6FyQZV/', 'https://www.instagram.com/p/DAIc5-eSp0G/', 'https://www.instagram.com/p/DAGS1XByGCx/', 'https://www.instagram.com/p/DAFxUQZSDTo/', 'https://www.instagram.com/p/DADl7hSyiyj/', 'https://www.instagram.com/p/DAAwn49RmYw/', 'https://www.instagram.com/p/C_-Xsl9ykQZ/', 'https://www.instagram.com/p/C_7GW1jSqg2/', 'https://www.instagram.com/p/C_5WQTCyDWl/', 'https://www.instagram.com/p/C_4HPztTfEM/', 'https://www.instagram.com/reel/C_21B9cSORC/']
12


In [ ]:
def get_dom_posts():
    post_links = []
    mainDiv = driver.find_element(By.XPATH,"//main[@role='main']")
    mainChildContainer = mainDiv.find_elements(By.XPATH,"./div") 
    print(len(mainChildContainer))
    postContainer = mainChildContainer[0].find_elements(By.XPATH,"./div")[-1]
    postStyleDiv = postContainer.find_elements(By.XPATH,"./div")[0]
    postRowContainer = postStyleDiv.find_elements(By.XPATH,"./div") 
    print(len(postRowContainer))

    for div_row in postRowContainer:
        divs = div_row.find_elements(By.XPATH, "div")
        # print(f"Number of <div> elements in each row: {len(divs)}")
        for div in divs:
            a_tags = div.find_elements(By.TAG_NAME, "a")
            if len(a_tags) > 0:
                a_tag = a_tags[0]
                href = a_tag.get_attribute("href")
                post_links.append(href)
    print(post_links)
    print(len(post_links))
    return post_links

In [ ]:
post_info = {}
post_links = get_dom_posts()


In [173]:
def parseNumber(strNumber):
    if '万' in strNumber:
        str_num = strNumber[0:-1]
        value = float(str_num)*10000
        return str(value)
    return strNumber

In [174]:
def parse_view(div_view_container,count_sperator):
    json_view = {}

    view_total = div_view_container.find_element(By.XPATH,"./div/div/div[2]/div/span")
    json_view['ビュー'] = parseNumber(view_total.text)

    per_follower = div_view_container.find_element(By.XPATH,"./div/div/div[3]/div[2]")
    json_view['フォロワー'] = per_follower.text

    per_unfollower = div_view_container.find_element(By.XPATH,"./div/div/div[3]/div[4]")
    json_view['フォロワー以外'] = per_unfollower.text

    if count_sperator == 7:
        view_source_div = div_view_container.find_element(By.XPATH,"./div/div/div[5]")
        view_source_contents = view_source_div.find_elements(By.XPATH,"./div")
        for content in view_source_contents:
            key_div = content.find_element(By.XPATH,"./div/div[1]/span")
            value_div = content.find_element(By.XPATH,"./div/div[2]/span")
            json_view[key_div.text] = value_div.text

    reach_count_div = div_view_container.find_element(By.XPATH,f"./div/div/div[{count_sperator}]")
    reach_key = reach_count_div.find_element(By.XPATH,"./span")
    reach_value = reach_count_div.find_element(By.XPATH,"./div/span")
    json_view[reach_key.text] = reach_value.text
    return json_view

In [175]:
def parse_interaction(div_interaction_container):
    json_interaction = {}
    div_children = div_interaction_container.find_elements(By.XPATH,'./div/div')
    interaction_container = div_children[1]
    post_interaction_container = div_children[3]
    action_account_container = div_children[5]

    # interaction
    interaction_key = interaction_container.find_element(By.XPATH,'./div/div[1]/span')
    interaction_value = interaction_container.find_element(By.XPATH,'./div/div[1]/div/span')
    json_interaction[interaction_key.text] = interaction_value.text
    # print(f'interaction key = {interaction_key.text}  interaction value = {interaction_value.text}')
    
    follower_key = interaction_container.find_element(By.XPATH,'./div/div[2]/div[1]/span')
    follower_value = interaction_container.find_element(By.XPATH,'./div/div[2]/div[2]')
    json_interaction[follower_key.text] = follower_value.text
    # print(f'follower key = {follower_key.text}  follower value = {follower_value.text}')
    
    unfollower_key = interaction_container.find_element(By.XPATH,'./div/div[2]/div[3]/span')
    unfollower_value = interaction_container.find_element(By.XPATH,'./div/div[2]/div[4]')
    json_interaction[unfollower_key.text] = unfollower_value.text
    # print(f'unfollower key = {unfollower_key.text}  unfollower value = {unfollower_value.text}')

    # post interaction
    post_children = post_interaction_container.find_elements(By.XPATH,'./div/div')
    post_interaction_div = post_children[0]
    post_interaction_key = post_interaction_div.find_element(By.XPATH,'./span')
    post_interaction_value = post_interaction_div.find_element(By.XPATH,'./div/span')
    json_interaction[post_interaction_key.text] = post_interaction_value.text
    # post_interaction_key = post_interaction_container.find_element(By.XPATH,'./div/div[1]/span')
    # post_interaction_value = post_interaction_container.find_element(By.XPATH,'./div/div[1]/div/span')
    # print(f'post_interaction key = {post_interaction_key.text}  post_interaction value = {post_interaction_value.text}')
    for element in post_children[1:]:
        key_div = element.find_element(By.XPATH,'./div/div[1]/span')
        value_div = element.find_element(By.XPATH,'./div/div[2]/span')
        json_interaction[key_div.text] = value_div.text
        # print(f'element key = {key_div.text}   element value = {value_div.text}')
    
    # action account
    action_account_key = action_account_container.find_element(By.XPATH,'./span')
    action_account_value = action_account_container.find_element(By.XPATH,'./div/span')
    # print(f'action account  key = {action_account_key.text}   value = {action_account_value.text}')
    json_interaction[action_account_key.text] = action_account_value.text

    return json_interaction
    

In [176]:
def parse_profile(div_profile_container):
    json_profile = {}
    profile_container = div_profile_container.find_element(By.XPATH,'./div/div[2]/div')
    children_elements = profile_container.find_elements(By.XPATH,'./div')
    profile_activity = children_elements[0]
    profile_activity_key = profile_activity.find_element(By.XPATH,'./span')
    profile_activity_value = profile_activity.find_element(By.XPATH,'./div/span')
    json_profile[profile_activity_key.text] = profile_activity_value.text
    for element in children_elements[1:]:
        key_div = element.find_element(By.XPATH,'./div[1]/span')
        value_div = element.find_element(By.XPATH,'./div[2]/span')
        json_profile[key_div.text] = value_div.text
    return json_profile

In [177]:
async def open_post_href(link):
    json_output = []
    home_window = driver.current_window_handle
    driver.switch_to.new_window('tab')
    driver.get(link)
    #
    time_tag = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.TAG_NAME, "time"))
    )
    date_time = time_tag.get_attribute("datetime")
    print(date_time)

    try:
        insight_button = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,"//div[@role='button' and text()='インサイトを見る']")))
        print(insight_button.text)
        print(insight_button.is_enabled())
        driver.execute_script("arguments[0].click();",insight_button)
        WebDriverWait(driver, 10).until(EC.url_contains('insights'))
                
        # find seperators
        hr_separators = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH,"//hr"))
        )
        count_sperator = len(hr_separators)
        if count_sperator == 7:
            main_separators = [hr_separators[2],hr_separators[5],hr_separators[6]]
        else:
            main_separators = [hr_separators[1],hr_separators[4],hr_separators[5]]
        div_separators = [hr.find_element(By.XPATH,"..") for hr in main_separators]
        div_info_root = div_separators[0].find_element(By.XPATH,"..")
        div_info_children = div_info_root.find_elements(By.XPATH,"./div")
        div_view_container = div_info_children[0]
        div_interaction_container = div_info_children[2]
        div_profile_container = div_info_children[4]


        json_output.append(parse_view(div_view_container,count_sperator))  
        json_output.append(parse_interaction(div_interaction_container)) 
        json_output.append(parse_profile(div_profile_container)) 
        
        print(json_output)
        driver.close()
        driver.switch_to.window(home_window)
        return json_output
    except TimeoutException:
        driver.close()
        driver.switch_to.window(home_window)
        return None
    

In [178]:
test = 0
for link in post_links:
    if test > 3:
        break
    print(f"href == {link}")
    test = test + 1
    insight_data = await open_post_href(link)

href == https://www.instagram.com/p/DALSUyhveyl/
url -- https://www.instagram.com/p/DALSUyhveyl/
2024-09-21T10:46:29.000Z
インサイトを見る
True
insight url -- https://www.instagram.com/insights/media/3461941332044278949/
ビュー = 11000.0
per_follower total フォロワー = 81%
per_unfollower total フォロワー以外 = 19%
profile length - 3
profile_activity key = プロフィールのアクティビティ   profile_activity value = 45
element key = プロフィールへのアクセス   element value = 44
element key = フォロー数   element value = 1
[{'ビュー': '11000.0', 'フォロワー': '81%', 'フォロワー以外': '19%', 'ホーム': '7026', '発見から': '1806', 'その他': '121', 'リーチしたアカウント数': '6783'}, {'インタラクション': '376', 'フォロワー': '67.7%', 'フォロワー以外': '32.3%', '投稿でのインタラクション': '0', '「いいね！」': '339', '保存数': '32', 'コメント': '2', 'シェア数': '0', 'アクションを実行したアカウント': '349'}, {'プロフィールのアクティビティ': '45', 'プロフィールへのアクセス': '44', 'フォロー数': '1'}]
href == https://www.instagram.com/p/DAM0D6FyQZV/
url -- https://www.instagram.com/p/DAM0D6FyQZV/
2024-09-21T02:52:37.000Z
インサイトを見る
True
insight url -- https://www.instagram.com/insights

-----------------------------------------------------------------

In [13]:
def request_posts(page_url,start_date,post_targets):
    response = requests.get(page_url)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        posts = data['data']
        # Print or process the data
        # for post in posts:
        #     print(f"ID: {post['id']}")
        #     print(f"Timestamp: {post['timestamp']}")
        last = posts[len(posts)-1]
        last_timestamp_str = last['timestamp']
        last_timestamp = datetime.strptime(last_timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
        print(last_timestamp)
        print(start_date)
        print(last_timestamp > start_date)
        if last_timestamp >= start_date:
            # concate
            post_targets = post_targets + posts
            # request next page
            paging = data['paging']
            next_page_url = paging['next']
            request_posts(next_page_url,start_date,post_targets)
        else:
            for post in posts:
                post_timestamp_str = post['timestamp']
                post_timestamp = datetime.strptime(post_timestamp_str, "%Y-%m-%dT%H:%M:%S%z")
                if post_timestamp > start_date:
                   post_targets.append(post) 
            return post_targets
    else:
        # Print the error if the request was not successful
        print(f"Error: {response.status_code}")
        print(response.json())

In [15]:
ACCESS_TOKEN = 'IGQWRNTExZAcEZAzV3FuU0g3NWJKMnhzRW9tZAkV4eUtHZA1JYZA0hWaEp6Y183MmJneHNSUEpSeklVQXBBQktzcnl5aEtPUDVSU0pwUjA2a3ZAqQnFmMjdRbGFPR3YzM0lyd3Q3ZAHo2b01CY1JhZA1lkbmxRbjVlVldfZA3cZD'
posts_url = f'https://graph.instagram.com/me/media?fields=id,timestamp&access_token={ACCESS_TOKEN}'
print(posts_url)
start_date_str = '2024-06-01'
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
timezone = pytz.timezone('UTC')
start_date = timezone.localize(start_date)
post_targets = []
list_target = request_posts(posts_url,start_date,post_targets)
print(list_target)


https://graph.instagram.com/me/media?fields=id,timestamp&access_token=IGQWRNTExZAcEZAzV3FuU0g3NWJKMnhzRW9tZAkV4eUtHZA1JYZA0hWaEp6Y183MmJneHNSUEpSeklVQXBBQktzcnl5aEtPUDVSU0pwUjA2a3ZAqQnFmMjdRbGFPR3YzM0lyd3Q3ZAHo2b01CY1JhZA1lkbmxRbjVlVldfZA3cZD
2024-07-30 01:45:55+00:00
2024-06-01 00:00:00+00:00
True
2024-07-05 03:00:00+00:00
2024-06-01 00:00:00+00:00
True
2024-06-14 10:11:16+00:00
2024-06-01 00:00:00+00:00
True
2024-05-20 11:15:48+00:00
2024-06-01 00:00:00+00:00
False
None


In [27]:
date_string = "2024-07-30T01:45:55+0000"
date_string_1 = "2024-07-30"

# Convert to a datetime object
dt_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S%z")
dt_object_1 = datetime.strptime(date_string_1, "%Y-%m-%d")

# Print the datetime object
print(dt_object)
print(dt_object_1)

2024-07-30 01:45:55+00:00
2024-07-30 00:00:00


In [ ]:
# Optional argument, if not specified will search path.
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/hodinkeejapan/")

# request facebook api to get post count
post_links = []

article_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.TAG_NAME, "article"))
)


def open_post_href(link):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    time_tag = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.TAG_NAME, "time"))
    )
    date_time = time_tag.get_attribute("datetime")
    print(date_time)
    sections = driver.find_elements(By.TAG_NAME,"section")
    print(len(sections))
    #
    # insight_div = driver.find_element(By.XPATH, "//section//section//div[@role='button' and text()='インサイトを見る']")
    # print(f'insight div == {insight_div}')
    #
    for section in sections:
        childDiv = section.find_element(By.XPATH,"//div[@role='button' and text()='インサイトを見る']")
        print(f'sction - childDiv text:  {childDiv.get_attribute('text')}')
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


print(article_element.get_property("className"))
div_row_elements = article_element.find_elements(By.XPATH, "./div/div/div")
# print(f"Number of Rows of <div> elements found: {len(div_row_elements)}")
for div_row in div_row_elements:
    divs = div_row.find_elements(By.XPATH, "./div")
    # print(f"Number of <div> elements in each row: {len(divs)}")
    for div in divs:
        a_tag = div.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        post_links.append(href)
print(f'post_links  = {post_links}')        
        
test = 0      
# for div in div_elements:
for link in post_links:
    if test > 2:
        break
    print(f"href == {link}")
    open_post_href(link)
    test = test + 1
    # time.sleep(6)

# time.sleep(5) # Let the user actually see something!

# driver.quit()

In [1]:
obj = {'key_ori':'value_ori'}
obj_target = {'key_1':'value_1','key_2':'value_2','key_3':'value_3'}
obj.update(obj_target)
obj

{'key_ori': 'value_ori',
 'key_1': 'value_1',
 'key_2': 'value_2',
 'key_3': 'value_3'}

In [3]:
print('key_1' in obj_target.keys())

True


In [ ]:
urls_1 = ['https://www.instagram.com/p/DAVmWMzSpAA/', 'https://www.instagram.com/p/DAVZYNYSHjH/', 'https://www.instagram.com/p/DASX6WFy5Ld/', 'https://www.instagram.com/p/DAQZoUDBe_u/', 'https://www.instagram.com/p/DAQNZ9rS4vv/', 'https://www.instagram.com/p/DANX_LfRGW4/', 'https://www.instagram.com/p/DALSUyhveyl/', 'https://www.instagram.com/p/DAM0D6FyQZV/', 'https://www.instagram.com/p/DAIc5-eSp0G/', 'https://www.instagram.com/p/DAGS1XByGCx/', 'https://www.instagram.com/p/DAFxUQZSDTo/', 'https://www.instagram.com/p/DADl7hSyiyj/']

urls_2 = ['https://www.instagram.com/p/DAVmWMzSpAA/', 'https://www.instagram.com/p/DAVZYNYSHjH/', 'https://www.instagram.com/p/DASX6WFy5Ld/', 'https://www.instagram.com/p/DAQZoUDBe_u/', 'https://www.instagram.com/p/DAQNZ9rS4vv/', 'https://www.instagram.com/p/DANX_LfRGW4/', 'https://www.instagram.com/p/DALSUyhveyl/', 'https://www.instagram.com/p/DAM0D6FyQZV/', 'https://www.instagram.com/p/DAIc5-eSp0G/', 'https://www.instagram.com/p/DAGS1XByGCx/', 'https://www.instagram.com/p/DAFxUQZSDTo/', 'https://www.instagram.com/p/DADl7hSyiyj/']

keys = ['https://www.instagram.com/p/DAVZYNYSHjH/', 'https://www.instagram.com/p/DASX6WFy5Ld/', 'https://www.instagram.com/p/DAQNZ9rS4vv/', 'https://www.instagram.com/p/DANX_LfRGW4/', 'https://www.instagram.com/p/DALSUyhveyl/', 'https://www.instagram.com/p/DAM0D6FyQZV/', 'https://www.instagram.com/p/DAIc5-eSp0G/', 'https://www.instagram.com/p/DAFxUQZSDTo/', 'https://www.instagram.com/p/DADl7hSyiyj/']

In [29]:
data = {'https://www.instagram.com/p/DAVZYNYSHjH/': {'Date': '2024-09-25 09:00:31', 'ビュー': '14000.0', 'ビューフォロワー': '94.8%', 'ビューフォロワー以外': '5.2%', 'ホーム': '1.1万', '発見から': '420', 'その他': '178', 'リーチしたアカウン ト数': '7304', 'インタラクション': '285', 'インタラクション フォロワー': '91.3%', 'インタラクション フォロワー以外': '8.7%', '投稿でのインタラクション': '0', '「いいね！」': '267', '保存数': '17', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '275', 'プロフィールのアクティビティ': '15', ' プロフィールへのアクセス': '13', 'フォロー数': '2'}, 'https://www.instagram.com/p/DASX6WFy5Ld/': {'Date': '2024-09-24 04:49:59', 'ビュー': '5611', 'ビューフォロワー': '89.6%', 'ビューフォロワー以外': '10.4%', 'ホーム': '4896', 'その他': '162', 'ハッシュタグ': '118', 'リーチしたアカウント数': '3466', 'インタラクション': '156', 'インタラクション フォロワー': '92.5%', 'インタラクション フォロワー以外': '7.5%', '投稿でのインタラクション': '0', '「いいね！」': '147', '保存数': '6', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウン ト': '156', 'プロフィールのアクティビティ': '12', 'プロフィールへのアクセス': '11', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/DAQNZ9rS4vv/': {'Date': '2024-09-23 08:39:42', 'ビュー': '7675', 'ビューフォロワー': '97.6%', 'ビューフォロワー以外': '2.4%', 'ホーム': '6334', 'プロフ ィール': '128', 'その他': '35', 'リーチしたアカウント数': '4227', 'インタラク ション': '94', 'インタラクション フォロワー': '100%', 'インタラクション フォロワー以外': '0%', '投稿でのインタラクション': '0', '「いいね！」': '91', '保存 数': '3', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '98', 'プロフィールのアクティビティ': '42', 'プロフィールへのアクセス': '32', '外部リンクのタップ': '10', 'フォロー数': '0'}, 'https://www.instagram.com/p/DANX_LfRGW4/': {'Date': '2024-09-22 06:14:27', 'ビュー': '16000.0', 'ビューフォロワー': '92.7%', 'ビューフォロワー以外': '7.3%', 'ホーム': '1万', '発見から': '752', 'その他': '316', 'リーチしたアカウント数': '7066', 'インタラクション': '341', 'インタラクション フォロワー': '85.2%', 'インタラクション フォロワー以外': '14.8%', '投稿でのインタラクション': '0', '「いいね！」': '329', '保存数': '11', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '330', 'プロフィールのアクティビティ': '39', 'プロフィールへのアクセス': '37', '外部リンクのタップ': '1', 'フォロー数': '1'}, 'https://www.instagram.com/p/DALSUyhveyl/': {'Date': '2024-09-21 10:46:29', 'ビュー': '21000.0', 'ビューフォロワー': '63%', 'ビューフォロワー以外': '37%', 'ホーム': '1万', '発見から': '6867', 'その他': '369', 'リーチしたアカウント数': '1.4万', 'インタラクション': '781', 'インタラクション フォロワー': '47.5%', 'インタラクション フォロワー以外': '52.5%', '投稿でのインタラクション': '0', '「いいね！」': '684', '保存数': '87', 'コメント': '4', 'シェア数': '0', 'アクションを実行したアカウント': '741', 'プロフィールのアクティビティ': '142', 'プロフィールへのアクセス': '127', ' 外部リンクのタップ': '2', 'フォロー数': '13'}, 'https://www.instagram.com/p/DAM0D6FyQZV/': {'Date': '2024-09-21 02:52:37', 'ビュー': '13000.0', 'ビューフォ ロワー': '91.3%', 'ビューフォロワー以外': '8.7%', 'ホーム': '9469', '発見から': '588', 'その他': '402', 'リーチしたアカウント数': '6789', 'インタラクション': '332', 'インタラクション フォロワー': '84.6%', 'インタラクション フォロワー 以外': '15.4%', '投稿でのインタラクション': '0', '「いいね！」': '294', '保存 数': '34', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '333', 'プロフィールのアクティビティ': '43', 'プロフィールへのアクセス': '39', '外部リンクのタップ': '1', 'フォロー数': '3'}, 'https://www.instagram.com/p/DAIc5-eSp0G/': {'Date': '2024-09-20 08:21:13', 'ビュー': '9885', 'ビューフォロワー': '94.1%', 'ビューフォロワー以外': '5.9%', 'ホーム': '7678', 'その他': '293', 'プロフィール': '254', 'リーチしたアカウント数': '5336', 'インタラクショ ン': '236', 'インタラクション フォロワー': '92.1%', 'インタラクション フォロワー以外': '7.9%', '投稿でのインタラクション': '0', '「いいね！」': '219', '保存数': '15', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '234', 'プロフィールのアクティビティ': '30', 'プロフィールへのアクセス': '25', '外部リンクのタップ': '3', 'フォロー数': '2'}, 'https://www.instagram.com/p/DAFxUQZSDTo/': {'Date': '2024-09-19 07:21:51', 'ビュー': '22000.0', 'ビューフ ォロワー': '62.6%', 'ビューフォロワー以外': '37.4%', 'ホーム': '1.2万', '発見 から': '6949', 'その他': '590', 'リーチしたアカウント数': '1.4万', 'インタラクション': '1168', 'インタラクション フォロワー': '52.4%', 'インタラクション フ ォロワー以外': '47.6%', '投稿でのインタラクション': '0', '「いいね！」': '1048', '保存数': '108', 'コメント': '4', 'シェア数': '0', 'アクションを実行したア カウント': '1098', 'プロフィールのアクティビティ': '95', 'プロフィールへのアクセス': '85', '外部リンクのタップ': '2', 'フォロー数': '8'}, 'https://www.instagram.com/p/DADl7hSyiyj/': {'Date': '2024-09-18 11:03:52', 'ビュー': '11000.0', 'ビューフォロワー': '86.9%', 'ビューフォロワー以外': '13.1%', 'ホーム': '8275', 'その他': '416', '発見から': '352', 'リーチしたアカウント数': '6416', 'インタラクション': '301', 'インタラクション フォロワー': '86.5%', 'インタラクショ ン フォロワー以外': '13.5%', '投稿でのインタラクション': '0', '「いいね！」': '279', '保存数': '17', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '285', 'プロフィールのアクティビティ': '22', 'プロフィールへのア クセス': '20', 'フォロー数': '2'}, 'https://www.instagram.com/p/DAAwn49RmYw/': {'Date': '2024-09-17 08:39:35', 'ビュー': '7881', 'ビューフォロワー': '94.7%', 'ビューフォロワー以外': '5.3%', 'ホーム': '6585', 'プロフィール': '270', 'その他': '223', 'リーチしたアカウント数': '6362', 'インタラクション': '119', 'インタラクション フォロワー': '92.6%', 'インタラクション フォロワー以外': '7.4%', '投稿でのインタラクション': '0', '「いいね！」': '109', '保存数': '10', 'コ メント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '116', 'プロ フィールのアクティビティ': '39', 'プロフィールへのアクセス': '34', '外部リンクのタップ': '5', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_-Xsl9ykQZ/': {'Date': '2024-09-16 10:23:18', 'ビュー': '13000.0', 'ビューフォロワー': '93.4%', 'ビューフォロワー以外': '6.6%', 'ホーム': '9307', 'プロフィール': '321', 'その他': '167', 'リーチしたアカウント数': '6566', 'インタラクション': '247', 'インタラクション フォロワー': '93.5%', 'インタラクション フォロワー以外': '6.5%', '投稿でのインタラクション': '0', '「いいね！」': '232', '保存数': '14', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '240', 'プロフィールのアクティビティ': '29', 'プロフィールへのアクセス': '28', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_7GW1jSqg2/': {'Date': '2024-09-15 03:54:04', 'ビュー': '14000.0', 'ビューフォロワー': '75.5%', 'ビューフォロワー以外': '24.5%', 'ホーム': '8869', '発見から': '1159', 'その他': '461', 'リーチしたアカウント数': '8173', 'インタラクション': '291', 'インタラクション フォロワー': '84.7%', 'インタラクション フォロワー以外': '15.3%', '投稿でのインタラクション': '0', '「いいね！」': '266', '保存数': '21', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '279', ' プロフィールのアクティビティ': '53', 'プロフィールへのアクセス': '48', '外部リンクのタップ': '4', 'フォロー数': '1'}, 'https://www.instagram.com/p/C_4HPztTfEM/': {'Date': '2024-09-14 00:52:18', 'ビュー': '15000.0', 'ビューフォロワー': '84.6%', 'ビューフォロワー以外': '15.4%', 'ホーム': '1.1万', '発見から': '856', 'その他': '553', 'リーチしたアカウント数': '9566', 'インタラクション': '502', 'インタラクション フォロワー': '88.5%', 'インタラクション フォロワー以外': '11.5%', '投稿でのインタラクション': '0', '「いいね！」': '465', '保存数': '25', 'コメント': '5', 'シェア数': '0', 'アクションを実行したアカウント': '493', 'プロフィールのアクティビティ': '59', 'プロフィールへのアクセス': '56', 'フォ ロー数': '3'}, 'https://www.instagram.com/reel/C_21B9cSORC/': {'Date': '2024-09-13 12:07:30', 'ビュー': '12000.0', 'ビュー フォロワー': '', 'ビュー フォロワー以外': '', 'リーチしたアカウント数': '8047', 'インタラクション': '228', 'イ ンタラクション フォロワー': '96.2%', 'インタラクション フォロワー以外': '3.8%', 'リールでのインタラクション': '0', '「いいね！」': '214', 'コメント': '0', '保存数': '7', 'シェア数': '0', 'アクションを実行したアカウント': '226', 'プロ フィールのアクティビティ': '0', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_ziIz4ytkc/': {'Date': '2024-09-12 05:22:53', 'ビュー': '16000.0', 'ビューフォロワー': '88.1%', 'ビューフォロワー以外': '11.9%', 'ホーム': '1.2万', 'プ ロフィール': '459', 'その他': '100', 'リーチしたアカウント数': '8956', 'インタラクション': '182', 'インタラクション フォロワー': '95.2%', 'インタラクション フォロワー以外': '4.8%', '投稿でのインタラクション': '0', '「いいね！」': '165', '保存数': '15', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '182', 'プロフィールのアクティビティ': '38', 'プロフィールへのアクセ ス': '32', '外部リンクのタップ': '3', 'フォロー数': '3'}, 'https://www.instagram.com/p/C_xhD9zyLab/': {'Date': '2024-09-11 10:35:00', 'ビュー': '18000.0', 'ビューフォロワー': '61%', 'ビューフォロワー以外': '39%', 'ホーム': '1万', '発 見から': '5836', 'その他': '570', 'リーチしたアカウント数': '1.3万', 'インタラクション': '444', 'インタラクション フォロワー': '54.5%', 'インタラクション フォロワー以外': '45.5%', '投稿でのインタラクション': '0', '「いいね！」': '414', '保存数': '28', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカ ウント': '439', 'プロフィールのアクティビティ': '74', 'プロフィールへのアクセ ス': '63', '外部リンクのタップ': '6', 'フォロー数': '5'}, 'https://www.instagram.com/reel/C_rzU3KSAzS/': {'Date': '2024-09-09 05:20:42', 'ビュー': '18000.0', 'ビュー フォロワー': '', 'ビュー フォロワー以外': '', 'リーチしたアカウント 数': '1.2万', 'インタラクション': '664', 'インタラクション フォロワー': '68.5%', 'インタラクション フォロワー以外': '31.5%', 'リールでのインタラクション': '0', '「いいね！」': '600', 'コメント': '5', '保存数': '54', 'シェア数': '0', 'アクションを実行したアカウント': '636', 'プロフィールのアクティビティ': '16', 'フォロー数': '16'}, 'https://www.instagram.com/p/C_pnG1Fx9tO/': {'Date': '2024-09-08 08:53:54', 'ビュー': '11000.0', 'ビューフォロワー': '88.3%', 'ビューフォロワー以外': '11.7%', 'ホーム': '8478', '発見から': '700', 'その他': '460', 'リーチしたアカウント数': '6466', 'インタラクション': '464', 'インタラクション フォロワー': '81.7%', 'インタラクション フォロワー以外': '18.3%', '投稿でのインタラクション': '0', '「いいね！」': '438', '保存数': '22', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '445', 'プロフィールのアク ティビティ': '35', 'プロフィールへのアクセス': '29', '外部リンクのタップ': '4', 'フォロー数': '2'}, 'https://www.instagram.com/p/C_m-SMHx95K/': {'Date': '2024-09-07 08:18:42', 'ビュー': '13000.0', 'ビューフォロワー': '83.4%', 'ビュー フォロワー以外': '16.6%', 'ホーム': '9315', '発見から': '1632', 'その他': '636', 'リーチしたアカウント数': '7828', 'インタラクション': '294', 'インタラクシ ョン フォロワー': '78.5%', 'インタラクション フォロワー以外': '21.5%', '投稿でのインタラクション': '0', '「いいね！」': '279', '保存数': '14', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '293', 'プロフィールの アクティビティ': '21', 'プロフィールへのアクセス': '19', '外部リンクのタップ': '2', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_klwgSSZgN/': {'Date': '2024-09-06 10:05:55', 'ビュー': '24000.0', 'ビューフォロワー': '57.9%', 'ビ ューフォロワー以外': '42.1%', 'ホーム': '1.3万', '発見から': '5731', 'その他': '883', 'リーチしたアカウント数': '1.6万', 'インタラクション': '1247', 'インタラクション フォロワー': '55.8%', 'インタラクション フォロワー以外': '44.2%', '投稿でのインタラクション': '0', '「いいね！」': '1081', '保存数': '152', 'コメント': '4', 'シェア数': '0', 'アクションを実行したアカウント': '1194', 'プロフィールのアクティビティ': '12', 'プロフィールへのアクセス': '1', 'フォロー数': '11'}, 'https://www.instagram.com/p/C_j10cDygQv/': {'Date': '2024-09-06 03:07:01', 'ビュー': '11000.0', 'ビューフォロワー': '78.7%', 'ビューフォロワー以外': '21.3%', 'ホーム': '4787', 'プロフィール': '635', 'その他': '67', 'リーチしたアカウント数': '6669', 'インタラクション': '86', 'インタラクション フォロワー': '95.4%', 'インタラクション フォロワー以外': '4.6%', '投稿でのインタラクショ ン': '0', '「いいね！」': '85', '保存数': '1', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '87', 'プロフィールのアクティビティ': '11', 'プロフィールへのアクセス': '11', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_h9txpya32/': {'Date': '2024-09-05 09:37:32', 'ビュー': '12000.0', ' ビューフォロワー': '87.4%', 'ビューフォロワー以外': '12.6%', 'ホーム': '8357', '発見から': '998', 'その他': '603', 'リーチしたアカウント数': '6693', 'インタラクション': '284', 'インタラクション フォロワー': '72.5%', 'インタラクション フォロワー以外': '27.5%', '投稿でのインタラクション': '0', '「いいね！」': '263', '保存数': '20', 'コメント': '1', 'シェア数': '0', 'アクションを実行したア カウント': '269', 'プロフィールのアクティビティ': '32', 'プロフィールへのアク セス': '31', 'フォロー数': '1'}, 'https://www.instagram.com/p/C_fjcqkSFwT/': {'Date': '2024-09-04 11:09:31', 'ビュー': '13000.0', 'ビューフォロワー': '73.8%', 'ビューフォロワー以外': '26.2%', 'ホーム': '7625', '発見から': '1024', 'そ の他': '668', 'リーチしたアカウント数': '8116', 'インタラクション': '286', 'インタラクション フォロワー': '78.5%', 'インタラクション フォロワー以外': '21.5%', '投稿でのインタラクション': '0', '「いいね！」': '268', '保存数': '17', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '291', 'プロ フィールのアクティビティ': '60', 'プロフィールへのアクセス': '56', '外部リンクのタップ': '2', 'フォロー数': '2'}, 'https://www.instagram.com/p/C_fWSwMStdQ/': {'Date': '2024-09-04 09:14:34', 'ビュー': '16000.0', 'ビューフォロワー': '71.7%', 'ビューフォロワー以外': '28.3%', 'ホーム': '1万', '発見から': '3319', ' その他': '848', 'リーチしたアカウント数': '1.1万', 'インタラクション': '734', 'インタラクション フォロワー': '71.4%', 'インタラクション フォロワー以外': '28.6%', '投稿でのインタラクション': '0', '「いいね！」': '669', '保存数': '62', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '707', 'プロフィールのアクティビティ': '75', 'プロフィールへのアクセス': '67', '外部リンクのタップ': '2', 'フォロー数': '6'}, 'https://www.instagram.com/p/C_cm7YiRyz8/': {'Date': '2024-09-03 07:42:13', 'ビュー': '15000.0', 'ビューフォロワー': '71.2%', 'ビューフォロワー以外': '28.8%', 'ホーム': '1万', '発見から': '3571', 'その他': '565', 'プロフィール': '514', 'リーチしたアカウント数': '1万', 'インタラクション': '443', 'インタラクション フォロワー': '68.1%', 'インタラクショ ン フォロワー以外': '31.9%', '投稿でのインタラクション': '0', '「いいね！」': '408', '保存数': '31', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '428', 'プロフィールのアクティビティ': '98', 'プロフィールへのア クセス': '89', '外部リンクのタップ': '7', 'フォロー数': '2'}, 'https://www.instagram.com/p/C_aRS1iy5LG/': {'Date': '2024-09-02 09:54:42', 'ビュー': '7773', 'ビューフォロワー': '89.7%', 'ビューフォロワー以外': '10.3%', 'ホーム': '6594', 'プロフィール': '737', 'その他': '279', 'リーチしたアカウント数': '5158', ' インタラクション': '254', 'インタラクション フォロワー': '95.3%', 'インタラク ション フォロワー以外': '4.7%', '投稿でのインタラクション': '0', '「いいね！」': '238', '保存数': '16', 'コメント': '0', 'シェア数': '0', 'アクションを実行 したアカウント': '257', 'プロフィールのアクティビティ': '21', 'プロフィールへ のアクセス': '18', '外部リンクのタップ': '3', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_XQrEgxN5u/': {'Date': '2024-09-01 05:51:32', 'ビュー': '8743', 'ビューフォロワー': '91.4%', 'ビューフォロワー以外': '8.6%', 'ホーム': '7639', 'プロフィール': '551', 'その他': '201', 'リーチしたアカウント数': '5207', 'インタラクション': '139', 'インタラクション フォロワー': '87.6%', 'インタラ クション フォロワー以外': '12.4%', '投稿でのインタラクション': '0', '「いいね ！」': '127', '保存数': '9', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '141', 'プロフィールのアクティビティ': '25', 'プロフィール へのアクセス': '23', '外部リンクのタップ': '2', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_UsgjTRWRM/': {'Date': '2024-08-31 05:57:03', 'ビュー': '10000.0', 'ビューフォロワー': '80.8%', 'ビューフォロワー以外': '19.2%', 'ホーム': '5903', 'プロフィール': '486', 'その他': '218', 'リーチしたアカウント数': '4505', 'インタラクション': '146', 'インタラクション フォロワー': '94%', 'イン タラクション フォロワー以外': '6%', '投稿でのインタラクション': '0', '「いいね！」': '140', '保存数': '3', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '147', 'プロフィールのアクティビティ': '0', 'プロフィールへのアクセス': '0', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_UYREeyQ5o/': {'Date': '2024-08-31 03:00:10', 'ビュー': '7537', 'ビューフォロワー': '90%', 'ビューフォロワー以外': '10%', 'ホーム': '6484', 'プロフィール': '498', 'その他': '298', '発見から': '257', 'リーチしたアカウント数': '4806', 'インタラクション': '240', 'インタラクション フォロワー': '90.6%', 'インタラクション フォロワー以外': '9.4%', '投稿でのインタラクション': '0', '「いいね！」': '232', '保存数': '8', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウン ト': '241', 'プロフィールのアクティビティ': '36', 'プロフィールへのアクセス': '29', '外部リンクのタップ': '6', 'フォロー数': '1'}, 'https://www.instagram.com/p/C_Srtqcy-yn/': {'Date': '2024-08-30 11:12:24', 'ビュー': '8978', 'ビューフォロワー': '94.2%', 'ビューフォロワー以外': '5.8%', 'ホーム': '5773', 'プロフ ィール': '555', 'その他': '107', 'リーチしたアカウント数': '4291', 'インタラクション': '199', 'インタラクション フォロワー': '95.5%', 'インタラクション フォロワー以外': '4.5%', '投稿でのインタラクション': '0', '「いいね！」': '192', '保存数': '7', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウン ト': '198', 'プロフィールのアクティビティ': '11', 'プロフィールへのアクセス': '10', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_P4hNgSYMN/': {'Date': '2024-08-29 09:05:48', 'ビュー': '13000.0', 'ビュ ーフォロワー': '79.3%', 'ビューフォロワー以外': '20.7%', 'ホーム': '8794', '発見から': '1901', 'その他': '757', 'リーチしたアカウント数': '8274', 'インタラ クション': '553', 'インタラクション フォロワー': '68%', 'インタラクション フォロワー以外': '32%', '投稿でのインタラクション': '0', '「いいね！」': '503', ' 保存数': '50', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '530', 'プロフィールのアクティビティ': '54', 'プロフィールへのアクセス': '50', '外部リンクのタップ': '1', 'フォロー数': '3'}, 'https://www.instagram.com/p/C_Nn7A1SpaW/': {'Date': '2024-08-28 12:02:17', 'ビュー': '12000.0', 'ビュ ーフォロワー': '92.6%', 'ビューフォロワー以外': '7.4%', 'ホーム': '9951', 'プ ロフィール': '653', 'その他': '265', 'リーチしたアカウント数': '7081', 'インタラクション': '285', 'インタラクション フォロワー': '93.8%', 'インタラクション フォロワー以外': '6.2%', '投稿でのインタラクション': '0', '「いいね！」': '276', '保存数': '8', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカ ウント': '283', 'プロフィールのアクティビティ': '41', 'プロフィールへのアクセ ス': '41', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_KadIjydDs/': {'Date': '2024-08-27 06:06:53', 'ビュー': '11000.0', 'ビューフォロワー': '86.9%', 'ビューフォロワー以外': '13.1%', 'ホーム': '8571', 'プロフィール': '856', 'その他': '817', 'リーチしたアカウント数': '6845', 'インタラクション': '255', 'インタラクション フォロワー': '85.6%', 'インタラクション フォロワー以外': '14.4%', '投稿でのインタラクション': '0', '「いいね！」': '245', '保存数': '9', 'コ メント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '258', 'プロ フィールのアクティビティ': '18', 'プロフィールへのアクセス': '17', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/C_ICQotRnDR/': {'Date': '2024-08-26 07:56:59', 'ビュー': '21000.0', 'ビューフォロワー': '88.4%', 'ビューフォロワー以外': '11.6%', 'ホーム': '1.7万', 'その他': '914', 'プロフィール': '781', 'リーチしたアカウント数': '1.2万', 'インタラクション': '698', 'インタラクション フォロワー': '92.4%', 'インタラクション フォロワー以外': '7.6%', '投稿でのインタラクション': '0', '「いいね！」': '629', '保存数': '68', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '676', 'プロフィールのアクティビティ': '135', 'プロフィールへのアクセス': '116', '外 部リンクのタップ': '14', 'フォロー数': '5'}, 'https://www.instagram.com/p/C_CqjmER_mE/': {'Date': '2024-08-24 05:53:40', 'ビュー': '18000.0', 'ビューフォロ ワー': '83.1%', 'ビューフォロワー以外': '16.9%', 'ホーム': '1.2万', 'プロフィ ール': '858', 'その他': '766', 'リーチしたアカウント数': '1万', 'インタラクシ ョン': '460', 'インタラクション フォロワー': '86.6%', 'インタラクション フォロワー以外': '13.4%', '投稿でのインタラクション': '0', '「いいね！」': '409', ' 保存数': '50', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '440', 'プロフィールのアクティビティ': '48', 'プロフィールへのアクセス': '39', '外部リンクのタップ': '4', 'フォロー数': '5'}, 'https://www.instagram.com/p/C_AjrKGSAsr/': {'Date': '2024-08-23 10:15:03', 'ビュー': '12000.0', 'ビュ ーフォロワー': '92.1%', 'ビューフォロワー以外': '7.9%', 'ホーム': '8302', 'プ ロフィール': '735', 'その他': '210', 'リーチしたアカウント数': '5857', 'インタラクション': '169', 'インタラクション フォロワー': '95.3%', 'インタラクション フォロワー以外': '4.7%', '投稿でのインタラクション': '0', '「いいね！」': '163', '保存数': '6', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカ ウント': '169', 'プロフィールのアクティビティ': '24', 'プロフィールへのアクセ ス': '23', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-98jHiyfFP/': {'Date': '2024-08-22 09:55:10', 'ビュー': '19000.0', 'ビューフォロワー': '90.1%', 'ビューフォロワー以外': '9.9%', 'ホーム': '1.1万', 'その他': '902', '発見から': '849', 'リーチしたアカウント数': '8022', 'インタラクション': '289', 'インタラクション フォロワー': '91.6%', 'インタラクション フォロワー以外': '8.4%', '投稿でのインタラクション': '0', '「いいね！」': '266', '保存数': '22', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '276', 'プロフィールのアクティビティ': '69', 'プロフィールへのアクセ ス': '62', '外部リンクのタップ': '3', 'フォロー数': '4'}, 'https://www.instagram.com/p/C-7ZHA2S8n9/': {'Date': '2024-08-21 10:06:31', 'ビュー': '6551', 'ビ ューフォロワー': '87.4%', 'ビューフォロワー以外': '12.6%', 'ホーム': '4377', 'プロフィール': '760', 'その他': '203', 'リーチしたアカウント数': '5200', 'インタラクション': '153', 'インタラクション フォロワー': '89.8%', 'インタラクショ ン フォロワー以外': '10.2%', '投稿でのインタラクション': '0', '「いいね！」': '146', '保存数': '6', 'コメント': '0', 'シェア数': '0', 'アクションを実行した アカウント': '154', 'プロフィールのアクティビティ': '14', 'プロフィールへのア クセス': '12', '外部リンクのタップ': '2', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-5B9HtysaY/': {'Date': '2024-08-20 12:05:43', 'ビュー': '14000.0', 'ビューフォロワー': '87.2%', 'ビューフォロワー以外': '12.8%', 'ホーム': '9865', '発見から': '1015', 'その他': '929', 'リーチしたアカウント数': '7864', ' インタラクション': '379', 'インタラクション フォロワー': '83.9%', 'インタラク ション フォロワー以外': '16.1%', '投稿でのインタラクション': '0', '「いいね！ 」': '348', '保存数': '28', 'コメント': '1', 'シェア数': '0', 'アクションを実 行したアカウント': '363', 'プロフィールのアクティビティ': '41', 'プロフィール へのアクセス': '36', '外部リンクのタップ': '3', 'フォロー数': '2'}, 'https://www.instagram.com/p/C-1rzLdR80X/': {'Date': '2024-08-19 04:54:23', 'ビュー': '7537', 'ビューフォロワー': '87.2%', 'ビューフォロワー以外': '12.8%', 'ホーム': '5245', 'プロフィール': '758', 'その他': '289', 'リーチしたアカウント数': '6066', 'インタラクション': '272', 'インタラクション フォロワー': '92.4%', 'インタラクション フォロワー以外': '7.6%', '投稿でのインタラクション': '0', '「いいね！」': '250', '保存数': '17', 'コメント': '3', 'シェア数': '0', 'アクションを 実行したアカウント': '258', 'プロフィールのアクティビティ': '39', 'プロフィー ルへのアクセス': '31', '外部リンクのタップ': '3', 'フォロー数': '5'}, 'https://www.instagram.com/p/C-zXsY5Rg9C/': {'Date': '2024-08-18 07:20:13', 'ビュー': '8233', 'ビューフォロワー': '86.2%', 'ビューフォロワー以外': '13.8%', 'ホーム': '6717', 'プロフィール': '681', '発見から': '597', 'その他': '238', 'リーチしたアカウント数': '5461', 'インタラクション': '330', 'インタラクション フォロワー': '87.6%', 'インタラクション フォロワー以外': '12.4%', '投稿でのインタラク ション': '0', '「いいね！」': '313', '保存数': '17', 'コメント': '0', 'シェア 数': '0', 'アクションを実行したアカウント': '320', 'プロフィールのアクティビティ': '20', 'プロフィールへのアクセス': '18', '外部リンクのタップ': '2', 'フォ ロー数': '0'}, 'https://www.instagram.com/p/C-xKJCUSA08/': {'Date': '2024-08-17 10:43:20', 'ビュー': '12000.0', 'ビューフォロワー': '90.5%', 'ビューフォロワー以外': '9.5%', 'ホーム': '9018', 'プロフィール': '714', 'その他': '608', 'リーチしたアカウント数': '6588', 'インタラクション': '305', 'インタラクション フォロワー': '88.5%', 'インタラクション フォロワー以外': '11.5%', '投稿でのイン タラクション': '0', '「いいね！」': '289', '保存数': '15', 'コメント': '0', ' シェア数': '0', 'アクションを実行したアカウント': '294', 'プロフィールのアクティビティ': '19', 'プロフィールへのアクセス': '16', '外部リンクのタップ': '3', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-up18zyPap/': {'Date': '2024-08-16 11:22:37', 'ビュー': '13000.0', 'ビューフォロワー': '88.4%', 'ビューフ ォロワー以外': '11.6%', 'ホーム': '1.1万', 'プロフィール': '1080', '発見から': '559', 'その他': '430', 'リーチしたアカウント数': '8121', 'インタラクション': '350', 'インタラクション フォロワー': '93.3%', 'インタラクション フォロワー以外': '6.7%', '投稿でのインタラクション': '0', '「いいね！」': '328', '保存数': '22', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '350', 'プロフィールのアクティビティ': '80', 'プロフィールへのアクセス': '68', ' 外部リンクのタップ': '12', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-rQ-ThyDqF/': {'Date': '2024-08-15 03:47:35', 'ビュー': '101000.0', 'ビューフォロワー': '25.3%', 'ビューフォロワー以外': '74.7%', 'ホーム': '4.8万', '発見か ら': '4.2万', 'その他': '1620', 'リーチしたアカウント数': '7.8万', 'インタラクション': '4277', 'インタラクション フォロワー': '35.1%', 'インタラクション フ ォロワー以外': '64.9%', '投稿でのインタラクション': '0', '「いいね！」': '3713', '保存数': '512', 'コメント': '21', 'シェア数': '0', 'アクションを実行したアカウント': '4251', 'プロフィールのアクティビティ': '1032', 'プロフィールへのアクセス': '788', '外部リンクのタップ': '25', 'フォロー数': '219'}, 'https://www.instagram.com/reel/C-pgcW-SfQT/': {'Date': '2024-08-14 11:24:45', 'ビュー': '13000.0', 'ビュー フォロワー': '', 'ビュー フォロワー以外': '', 'リーチしたア カウント数': '7584', 'インタラクション': '257', 'インタラクション フォロワー': '87.1%', 'インタラクション フォロワー以外': '12.9%', 'リールでのインタラクシ ョン': '0', '「いいね！」': '243', 'コメント': '2', '保存数': '12', 'シェア数': '0', 'アクションを実行したアカウント': '275', 'プロフィールのアクティビティ': '4', 'フォロー数': '4'}, 'https://www.instagram.com/p/C-m2FU2SdLI/': {'Date': '2024-08-13 10:35:39', 'ビュー': '38000.0', 'ビューフォロワー': '34.3%', 'ビューフォロワー以外': '65.7%', '発見から': '2.2万', 'ホーム': '1.2万', 'その他': '1182', 'リーチしたアカウント数': '3.1万', 'インタラクション': '1679', 'インタラクション フォロワー': '35.1%', 'インタラクション フォロワー以外': '64.9%', '投稿でのインタラクション': '0', '「いいね！」': '1495', '保存数': '182', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '1673', 'プロフィールのアクティビティ': '304', 'プロフィールへのアクセス': '247', '外部リンクのタップ': '11', 'フォロー数': '46'}, 'https://www.instagram.com/p/C-kJyMfymrO/': {'Date': '2024-08-12 09:30:05', 'ビュー': '13000.0', 'ビューフォロワー': '73.4%', 'ビューフォロワー以外': '26.6%', 'ホーム': '7999', '発見から': '2817', 'その他': '966', 'リーチしたアカウント数': '8741', 'インタラクション': '360', 'インタラクション フォロワー': '74.8%', 'インタラクション フォロワー以外': '25.2%', '投稿でのインタラクション': '0', '「いいね！」': '336', '保存数': '20', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '347', 'プロフィールのアクティビティ': '90', 'プロフィールへのアクセス': '74', '外部 リンクのタップ': '9', 'フォロー数': '7'}, 'https://www.instagram.com/p/C-h5t1ByQk0/': {'Date': '2024-08-11 12:31:11', 'ビュー': '9964', 'ビューフォロワー': '95%', 'ビューフォロワー以外': '5%', 'ホーム': '7244', 'プロフィール': '587', 'その他': '147', 'リーチしたアカウント数': '5280', 'インタラクション': '122', 'インタラクション フォロワー': '99.2%', 'インタラクション フォロワー以外': '0.8%', '投稿でのインタラクション': '0', '「いいね！」': '120', '保存数': '2', ' コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '122', 'プ ロフィールのアクティビティ': '9', 'プロフィールへのアクセス': '9', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-fdef8S7CV/': {'Date': '2024-08-10 13:45:57', 'ビュー': '14000.0', 'ビューフォロワー': '93.5%', 'ビューフォロワー以外': '6.5%', 'ホーム': '1.2万', 'プロフィール': '624', 'その他': '180', 'リーチ したアカウント数': '8538', 'インタラクション': '293', 'インタラクション フォロワー': '98.3%', 'インタラクション フォロワー以外': '1.7%', '投稿でのインタラクション': '0', '「いいね！」': '251', '保存数': '38', 'コメント': '0', 'シェア 数': '0', 'アクションを実行したアカウント': '288', 'プロフィールのアクティビティ': '87', 'プロフィールへのアクセス': '81', '外部リンクのタップ': '4', 'フォ ロー数': '2'}, 'https://www.instagram.com/p/C-aemPUyxzW/': {'Date': '2024-08-08 15:19:33', 'ビュー': '15000.0', 'ビューフォロワー': '69.7%', 'ビューフォロワー以外': '30.3%', 'ホーム': '9948', '発見から': '2053', 'その他': '984', 'リーチしたアカウント数': '1万', 'インタラクション': '763', 'インタラクション フォ ロワー': '73.5%', 'インタラクション フォロワー以外': '26.5%', '投稿でのインタ ラクション': '0', '「いいね！」': '671', '保存数': '78', 'コメント': '2', 'シ ェア数': '0', 'アクションを実行したアカウント': '719', 'プロフィールのアクティビティ': '113', 'プロフィールへのアクセス': '100', '外部リンクのタップ': '9', 'フォロー数': '4'}, 'https://www.instagram.com/p/C-Z4meRSABr/': {'Date': '2024-08-08 09:47:31', 'ビュー': '20000.0', 'ビューフォロワー': '68%', 'ビューフォ ロワー以外': '32%', 'ホーム': '1.5万', 'その他': '835', 'プロフィール': '811', 'リーチしたアカウント数': '1.3万', 'インタラクション': '620', 'インタラクション フォロワー': '91.6%', 'インタラクション フォロワー以外': '8.4%', '投稿での インタラクション': '0', '「いいね！」': '561', '保存数': '49', 'コメント': '2', 'シェア数': '0', 'アクションを実行したアカウント': '609', 'プロフィールのア クティビティ': '67', 'プロフィールへのアクセス': '63', '外部リンクのタップ': '1', 'フォロー数': '3'}, 'https://www.instagram.com/p/C-W-ioeR33c/': {'Date': '2024-08-07 06:41:42', 'ビュー': '15000.0', 'ビューフォロワー': '81.2%', 'ビュ ーフォロワー以外': '18.8%', 'ホーム': '1.1万', '発見から': '2185', 'その他': '970', 'リーチしたアカウント数': '9778', 'インタラクション': '479', 'インタラクション フォロワー': '76.4%', 'インタラクション フォロワー以外': '23.6%', '投稿でのインタラクション': '0', '「いいね！」': '434', '保存数': '42', 'コメント': '2', 'シェア数': '0', 'アクションを実行したアカウント': '469', 'プロフィール のアクティビティ': '78', 'プロフィールへのアクセス': '69', '外部リンクのタップ': '5', 'フォロー数': '4'}, 'https://www.instagram.com/p/C-U0QlHvo5a/': {'Date': '2024-08-06 10:33:23', 'ビュー': '10000.0', 'ビューフォロワー': '86.6%', ' ビューフォロワー以外': '13.4%', 'ホーム': '7653', 'プロフィール': '1043', 'そ の他': '459', 'リーチしたアカウント数': '6637', 'インタラクション': '307', 'インタラクション フォロワー': '86.5%', 'インタラクション フォロワー以外': '13.5%', '投稿でのインタラクション': '0', '「いいね！」': '290', '保存数': '14', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '306', 'プロ フィールのアクティビティ': '23', 'プロフィールへのアクセス': '22', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-SR_4ASVDJ/': {'Date': '2024-08-05 10:55:31', 'ビュー': '13000.0', 'ビューフォロワー': '66.3%', 'ビューフォロワー以外': '33.7%', 'ホーム': '7289', '発見から': '3153', 'その他': '1036', 'リーチしたアカウント数': '9251', 'インタラクション': '523', 'インタラクション フォロワー': '56.7%', 'インタラクション フォロワー以外': '43.3%', '投稿でのインタラクション': '0', '「いいね！」': '456', '保存数': '52', 'コメント': '3', 'シェア数': '0', 'アクションを実行したアカウント': '499', 'プロフィールのアクティビティ': '73', 'プロフィールへのアクセス': '58', '外部リンクのタップ': '3', 'フォロー数': '12'}, 'https://www.instagram.com/p/C-PfWqSRLD7/': {'Date': '2024-08-04 08:54:30', 'ビュー': '13000.0', 'ビューフォロワー': '85.5%', 'ビューフォロワー以外': '14.5%', 'ホーム': '9850', 'プロフィール': '1082', 'その他': '733', 'リーチしたアカウント数': '8040', 'インタラクション': '326', 'インタラクション フォロワー': '83.6%', 'インタラクション フォロワー以外': '16.4%', '投稿でのインタラクション': '0', '「いいね！」': '292', '保存数': '32', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '314', 'プロフィールのアクティビティ': '12', 'プロフィールへのアクセス': '10', '外部リンクのタップ': '1', 'フォロー数': '1'}, 'https://www.instagram.com/p/C-NGQTuywaA/': {'Date': '2024-08-03 10:36:42', 'ビュー': '15000.0', 'ビューフォロワー': '87.1%', 'ビューフォロワー以外': '12.9%', 'ホーム': '1.1万', '発見から': '1192', 'その他': '993', 'リーチしたアカウント数': '8187', 'インタラクション': '386', 'インタラクション フォロワー': '89.5%', 'インタラクション フォロワー以外': '10.5%', '投稿でのインタラクション': '0', '「いいね！」': '355', '保存数': '21', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '359', 'プロフィールのアクティビティ': '74', 'プロフィールへのアクセス': '67', '外部リンクのタップ': '3', 'フォロー数': '4'}, 'https://www.instagram.com/p/C-KYGR6SRjT/': {'Date': '2024-08-02 09:14:54', 'ビュー': '11000.0', 'ビューフォロワー': '59.7%', 'ビューフォロワー以外': '40.3%', 'ホーム': '5711', 'プロフィール': '1023', 'その他': '547', 'リーチしたアカウント数': '6558', 'インタラクシ ョン': '245', 'インタラクション フォロワー': '81.4%', 'インタラクション フォロワー以外': '18.6%', '投稿でのインタラクション': '0', '「いいね！」': '227', ' 保存数': '14', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '236', 'プロフィールのアクティビティ': '21', 'プロフィールへのアクセス': '21', 'フォロー数': '0'}, 'https://www.instagram.com/p/C-H7smOvsyo/': {'Date': '2024-08-01 10:28:15', 'ビュー': '9197', 'ビューフォロワー': '87.4%', 'ビューフォロワー以外': '12.6%', 'ホーム': '5523', 'プロフィール': '898', 'その他': '325', 'リーチしたアカウント数': '4806', 'インタラクション': '185', 'インタラクション フォロワー': '85.6%', 'インタラクション フォロワー以外': '14.4%', '投稿でのインタラクション': '0', '「いいね！」': '176', '保存数': '8', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '191', 'プロフィールのアクティビティ': '24', 'プロフィールへのアクセス': '22', '外部リンクのタップ': '2', 'フォロー数': '0'}}
type(data)

dict

In [30]:
# format value 
for url,postData in data.items():
    for key,value in postData.items():
        if '万' in value:
            str_num = value[0:-1]
            str_value = int(float(str_num)*10000)
            postData.update({key:str_value})
print(data)

{'https://www.instagram.com/p/DAVZYNYSHjH/': {'Date': '2024-09-25 09:00:31', 'ビュー': '14000.0', 'ビューフォロワー': '94.8%', 'ビューフォロワー以外': '5.2%', 'ホーム': 11000, '発見から': '420', 'その他': '178', 'リーチしたアカウン ト数': '7304', 'インタラクション': '285', 'インタラクション フォロワー': '91.3%', 'インタラクション フォロワー以外': '8.7%', '投稿でのインタラクション': '0', '「いいね！」': '267', '保存数': '17', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '275', 'プロフィールのアクティビティ': '15', ' プロフィールへのアクセス': '13', 'フォロー数': '2'}, 'https://www.instagram.com/p/DASX6WFy5Ld/': {'Date': '2024-09-24 04:49:59', 'ビュー': '5611', 'ビューフォロワー': '89.6%', 'ビューフォロワー以外': '10.4%', 'ホーム': '4896', 'その他': '162', 'ハッシュタグ': '118', 'リーチしたアカウント数': '3466', 'インタラクション': '156', 'インタラクション フォロワー': '92.5%', 'インタラクション フォロワー以外': '7.5%', '投稿でのインタラクション': '0', '「いいね！」': '147', '保存数': '6', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウン ト': '156', 'プロフィールのアクティビティ': '12', 'プロフィールへのアクセス': '11', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/DAQNZ9rS4vv/': {'Date': '2024-09-23 08:39:42', 'ビュー': '7675', 'ビューフ

In [31]:
def remove_spaces_from_keys(d):
    new_dict = {}
    for key, value in d.items():
        # Remove spaces in the key
        new_key = key.replace(' ', '')
        # If the value is a dictionary, apply the function recursively
        if isinstance(value, dict):
            new_dict[new_key] = remove_spaces_from_keys(value)
        else:
            new_dict[new_key] = value
    return new_dict

# Apply the function to the data
data_no_spaces = remove_spaces_from_keys(data)
print(data_no_spaces)

{'https://www.instagram.com/p/DAVZYNYSHjH/': {'Date': '2024-09-25 09:00:31', 'ビュー': '14000.0', 'ビューフォロワー': '94.8%', 'ビューフォロワー以外': '5.2%', 'ホーム': 11000, '発見から': '420', 'その他': '178', 'リーチしたアカウント数': '7304', 'インタラクション': '285', 'インタラクションフォロワー': '91.3%', 'インタラクションフォロワー以外': '8.7%', '投稿でのインタラクション': '0', '「いいね！」': '267', '保存数': '17', 'コメント': '1', 'シェア数': '0', 'アクションを実行したアカウント': '275', 'プロフィールのアクティビティ': '15', 'プロフィールへのアクセス': '13', 'フォロー数': '2'}, 'https://www.instagram.com/p/DASX6WFy5Ld/': {'Date': '2024-09-24 04:49:59', 'ビュー': '5611', 'ビューフォロワー': '89.6%', 'ビューフォロワー以外': '10.4%', 'ホーム': '4896', 'その他': '162', 'ハッシュタグ': '118', 'リーチしたアカウント数': '3466', 'インタラクション': '156', 'インタラクションフォロワー': '92.5%', 'インタラクションフォロワー以外': '7.5%', '投稿でのインタラクション': '0', '「いいね！」': '147', '保存数': '6', 'コメント': '0', 'シェア数': '0', 'アクションを実行したアカウント': '156', 'プロフィールのアクティビティ': '12', 'プロフィールへのアクセス': '11', '外部リンクのタップ': '1', 'フォロー数': '0'}, 'https://www.instagram.com/p/DAQNZ9rS4vv/': {'Date': '2024-09-23 08:39:42', 'ビュー': '7675', 'ビューフォロワー': 

In [40]:
import pytz
from datetime import datetime
utc = pytz.UTC
jst = pytz.timezone('Asia/Tokyo')
def convert_date_to_jst(data_dict):
    for url, details in data_dict.items():
        date_str = details.get('Date')
        if date_str:
            try:
                # Parse the date string as UTC and convert to JST
                utc_date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=utc)
                jst_date = utc_date.astimezone(jst)
                # Update the Date in the dictionary
                data_dict[url]['Date'] = jst_date.strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                pass  # If the date format is invalid, skip it
    return data_dict

# Apply the function to your data
data_updated = convert_date_to_jst(data_no_spaces)

In [32]:
def get_post_keys(data):
    key_list = []
    for _,data in data.items():
        keys = list(data.keys())
        for key in keys:
            if key not in key_list:
                key_list.append(key)
    return key_list

In [41]:
import csv
all_keys = get_post_keys(data_no_spaces)
print(all_keys)


# Step 2: Write to CSV
with open('output_2.csv', mode='w', newline='',encoding='utf-8') as file:
    fieldnames = ['URL'] + all_keys  # Header includes 'URL' and all keys
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the rows
    for url, values in data_no_spaces.items():
        # Prepare a row dict with URL and ensure all keys are accounted for with empty string if missing
        row = {'URL': url}
        row.update({key: values.get(key, '') for key in all_keys})  # Fill in missing keys with empty string
        writer.writerow(row)

['Date', 'ビュー', 'ビューフォロワー', 'ビューフォロワー以外', 'ホーム', '発見から', 'その他', 'リーチしたアカウント数', 'インタラクション', 'インタラクションフォロワー', 'インタラクションフォロワー以外', '投稿でのインタラクション', '「いいね！」', '保存数', 'コメント', 'シェア数', 'アクションを実行したアカウント', 'プロフィールのアクティビティ', 'プロフィールへのアクセス', 'フォロー数', 'ハッシュタグ', '外部リンクのタップ', 'プロフィール', 'リールでのインタラクション']


In [34]:
def get_column_letter(n):
    """ Convert a column index (1-based) into an alphabetic column label (e.g., 1 -> A, 27 -> AA). """
    result = ''
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

In [42]:
import gspread
from google_auth_oauthlib.flow import InstalledAppFlow
SCOPES = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
def save_to_spreadsheet():
    # print(f'obj_post_data --\n{data_no_spaces}')
    # OAuth 2.0 flow to get the credentials
    flow = InstalledAppFlow.from_client_secrets_file('client_secret.json',SCOPES)
    
    # Authenticate using the credentials.json
    creds = flow.run_local_server(port=0)

    # Authorize the client
    client = gspread.authorize(creds)

    # Open the Google Spreadsheet (replace 'spreadsheet_name' with the actual name)
    # spreadsheet = client.open('Instagramインサイトデータ更新')
    spreadsheet = client.open("test_instagram")

    # Select a specific sheet by its name (or use .sheet1 for the first sheet)
    sheet = spreadsheet.worksheet("hodinkee")
    
    # Step 2: Read the CSV file
    with open('output_2.csv', 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Convert CSV data to a list
        csv_data = list(csv_reader)

    # Step 3: Upload data to Google Sheets
    sheet.update('A1', csv_data)
    pass   
save_to_spreadsheet()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=403240057380-iic1c3qso7ruo6smp58464bq59qf6c0b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63749%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=iEFA6JuYDIOeqsPdtDSkdcTMZKS3o3&access_type=offline


C:\Users\ddadmin\AppData\Local\Temp\ipykernel_17228\3340964502.py:30: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', csv_data)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=403240057380-iic1c3qso7ruo6smp58464bq59qf6c0b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59017%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=eZNranhoO5n1NNpuFSy3AJTu6yw9aj&access_type=offline


C:\Users\ddadmin\AppData\Local\Temp\ipykernel_17228\1368973962.py:30: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', csv_data)


In [32]:
import pandas as pd
df = pd.read_csv('output_1.csv', encoding='utf-8')
print(df.shape)
df.head(3)

(58, 25)


,URL,Date,ビュー,ビューフォロワー,ビューフォロワー以外,ホーム,発見から,その他,リーチしたアカウント数,インタラクション,インタラクションフォロワー,インタラクションフォロワー以外,投稿でのインタラクション,「いいね！」,保存数,コメント,シェア数,アクションを実行したアカウント,プロフィールのアクティビティ,プロフィールへのアクセス,フォロー数,ハッシュタグ,外部リンクのタップ,プロフィール,リールでのインタラクション
0,https://www.instagram.com/p/DAVZYNYSHjH/,2024-09-25 09:00:31,14000.0,94.8%,5.2%,1.1万,420,178.0,7304,285,91.3%,8.7%,0.0,267,17,1,0,275,15,13.0,2,NaN,NaN,NaN,NaN
1,https://www.instagram.com/p/DASX6WFy5Ld/,2024-09-24 04:49:59,5611.0,89.6%,10.4%,4896,NaN,162.0,3466,156,92.5%,7.5%,0.0,147,6,0,0,156,12,11.0,0,118.0,1.0,NaN,NaN
2,https://www.instagram.com/p/DAQNZ9rS4vv/,2024-09-23 08:39:42,7675.0,97.6%,2.4%,6334,NaN,35.0,4227,94,100%,0%,0.0,91,3,0,0,98,42,32.0,0,NaN,10.0,128.0,NaN
